# FABM box model

This notebook runs one of the CMEMS biogeochemical models
in a well-mixed box (0D), under constant environmental forcing

In [ ]:
# Python standard library
import os
import glob

# 3rd party packages
import ipywidgets
import numpy as np
import scipy.integrate

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (interactive but deprecated) or "%matplotlib inline" (static plots)
%matplotlib widget
import matplotlib.pyplot

# FABM itself
import pyfabm

In [ ]:
# Show available test cases
testcases = [(os.path.basename(path)[:-5], path) for path in glob.glob('../testcases/*.yaml')]
testcases.sort(key=lambda x: x[0])
testcase_dropdown = ipywidgets.RadioButtons(options=testcases, description='Test case:')
display(testcase_dropdown)

In [ ]:
# Initialize the test case (this reads fabm.yaml)
model = pyfabm.Model(testcase_dropdown.value)

# Present configurable environmental conditions
bottom_depth = 20.0
default_environment = dict(
    temperature = 15.0,
    practical_salinity = 35.0,
    surface_downwelling_shortwave_flux = 150.0,
    downwelling_shortwave_flux = 30.0,
    surface_downwelling_photosynthetic_radiative_flux = 50.0,
    downwelling_photosynthetic_radiative_flux = 10.0,
    density = 1020.0,
    mole_fraction_of_carbon_dioxide_in_air = 414.2,
    wind_speed = 2.0,
    surface_air_pressure = 101325,
    surface_temperature = 15.0,
    vertical_tracer_diffusivity = 1e-6,
    bottom_depth = bottom_depth,
    depth = 0.5 * bottom_depth,
    pressure = 0.5 * bottom_depth,
    cell_thickness = bottom_depth
)
model.cell_thickness = bottom_depth  # cell thickness in m, used by getRates to scale surface and bottom fluxes
labels, inputs, units, dependencies = [], [], [], []
daynr = None
for variable in model.dependencies:
    if variable.name == 'number_of_days_since_start_of_the_year':
        # Keep reference to day-of-the-year, so we can update it during the simulation
        daynr = variable
        daynr.value = 0.0
    else:
        # Present this environmental dependency as a configurable parameter,
        # to be kept constant during the simulation
        labels.append(ipywidgets.Label('%s:' % variable.long_name))
        inputs.append(ipywidgets.FloatText(value=default_environment.get(variable.name, 0.), layout={'width': '7em'}))
        units.append(ipywidgets.Label('%s' % variable.units))
        dependencies.append(variable)
display(ipywidgets.HBox((
    ipywidgets.VBox([ipywidgets.HTML('<b>Variable</b>')] + labels),
    ipywidgets.VBox([ipywidgets.HTML('<b>Value</b>')] + inputs),
    ipywidgets.VBox([ipywidgets.HTML('<b>Units</b>')] + units)
)))

results = []   # this list will contain results of repeated simulations

In [ ]:
# Transfer environmental conditions to model
for variable, widget in zip(dependencies, inputs):
    variable.value = widget.value

# Initialize model (this also verifies that all dependencies have been fulfilled)
assert model.start(), 'Model failed to start: %s' % pyfabm.getError()

# Backup initial state (to restore for repeated simulation)
initial_state = np.array(model.state)

# Time derivative
def dy(t, y):
    # Update the environment to the current time
    if daynr is not None:
        daynr.value = t

    # Obtain sinks and sources
    # (FABM rates are per second and converted here to per day)
    model.state[:] = y
    return model.getRates(t) * 86400

# Time-integrate over 200 days
times = np.arange(0, 200.0, 1)
result = scipy.integrate.solve_ivp(dy, [0., times[-1]], model.state, t_eval=times, max_step=0.1)
model.state[:] = initial_state
results.append(result)

# Plot results
matplotlib.pyplot.ioff()
fig, ax = matplotlib.pyplot.subplots()
lines = []
for result in results:
    line, = ax.plot(result.t, result.y[0, :])
    lines.append(line)
ax.grid()
ax.set_xlabel('time (d)')

def update(variable: int):
    v = model.state_variables[variable]
    for result, line in zip(results, lines):
        line.set_ydata(result.y[variable, :])
    ax.set_ylabel('%s (%s)' % (v.long_name, v.units))
    ax.set_title(v.long_path)
    ax.relim()
    ax.autoscale()
    fig.canvas.draw()
dropdown = ipywidgets.interactive(update, variable=[(variable.long_path, i) for i, variable in enumerate(model.state_variables)])
display(dropdown)
fig.show()